In [2]:
from collections import Counter
import logging
import re
import json
import jieba
from tqdm import tqdm

In [2]:
logger = logging.getLogger('les2')
logger.setLevel(logging.INFO)
console_handler = logging.StreamHandler()
# console_handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)

In [45]:
with open('./data/question.json', 'r', encoding='utf-8') as f:
    train_set = json.loads(f.read())

In [27]:
# 训练集共20000篇文章
len(train_set)

20000

In [5]:
def precision_recall_f1(prediction, ground_truth):
    if not isinstance(prediction, list):
        prediction_tokens = prediction.split()
    else:
        prediction_tokens = prediction
    if not isinstance(ground_truth, list):
        ground_truth_tokens = ground_truth.split()
    else:
        ground_truth_tokens = ground_truth
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0, 0, 0
    p = 1.0 * num_same / len(prediction_tokens)
    r = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * p * r) / (p + r)
    return p, r, f1

In [6]:
def recall(prediction, ground_truth):
    return precision_recall_f1(prediction, ground_truth)[1]


def f1_score(prediction, ground_truth):
    return precision_recall_f1(prediction, ground_truth)[2]

In [7]:
def metric_max_over_ground_truths(metric_fn, prediction, ground_truth):
    score = metric_fn(prediction, ground_truth)
    return score

In [8]:
# 找到最相关的段落和在段落中的位置
def find_fake_answer(sample):
    for a_idx, answer_token in enumerate(sample['questions']):
        most_related_para = -1
        most_related_para_len = 999999
        max_related_score = 0
#         print('a_idx=',a_idx, 'answer_token=',answer_token)
        for p_idx, para_tokens in enumerate(sample['segmented_article_content']):
            related_score = metric_max_over_ground_truths(recall,
                                                          para_tokens,
                                                          answer_token['segmented_answer'])
#             print('p_idx=',p_idx,'related_score=',related_score)
            if related_score > max_related_score \
                    or (related_score == max_related_score
                        and len(para_tokens) < most_related_para_len):
                most_related_para = p_idx
                most_related_para_len = len(para_tokens)
                max_related_score = related_score
        sample['questions'][a_idx]['most_related_para'] = most_related_para
        most_related_para_tokens = sample['segmented_article_content'][most_related_para]
        
        answer_tokens = set(answer_token['segmented_answer'])
        best_match_score = 0
        best_match_span = [-1, -1]
        best_fake_answer = None
        
        for start_tidx in range(len(most_related_para_tokens)):
            if most_related_para_tokens[start_tidx] not in answer_tokens:
                continue
            for end_tidx in range(len(most_related_para_tokens) - 1, start_tidx - 1, -1):
                span_tokens = most_related_para_tokens[start_tidx: end_tidx + 1]
                match_score = metric_max_over_ground_truths(f1_score, span_tokens,
                                                                answer_token['segmented_answer'])
                if match_score == 0:
                    break
                if match_score > best_match_score:
                    best_match_span = [start_tidx, end_tidx]
                    best_match_score = match_score
                    best_fake_answer = ''.join(span_tokens)
        sample['questions'][a_idx]['answer_spans'] = best_match_span
        sample['questions'][a_idx]['fake_answers'] = best_fake_answer
        sample['questions'][a_idx]['match_scores'] = best_match_score
    return sample

In [84]:
def clean_data(sample):
    # 文章内容和标题分段->分词：将标题插入到分段后的首位置
    sample['segmented_article_title'] = \
        list(jieba.cut(''.join(re.split(r'\u3000+|\s+|\t+',sample['article_title'].strip()))))
    
    sample_splited_para = re.split(r'\u3000+|\s+|\t+',sample['article_content'].strip())
    if len(sample_splited_para) == 1 and len(sample_splited_para[0]) > 200:
        sample_splited_para = re.split(r'\。',sample['article_content'].strip())
    sample_splited_list = []
    for para in sample_splited_para:
        sample_splited_list.append(list(jieba.cut(para.strip(), cut_all=False)))
    sample_splited_list.insert(0, sample['segmented_article_title'])

    sample['segmented_article_content'] = sample_splited_list
       
    # 问题和答案分词处理
    for i,question in enumerate(sample['questions']):
        sample['questions'][i]['segmented_question'] = \
            list(jieba.cut(''.join(question['question'].strip().split('\u3000+|\s+|\t+'))))
        sample['questions'][i]['segmented_answer'] = \
            list(jieba.cut(''.join(question['answer'].strip().split('\u3000+|\s+|\t+'))))
    return sample

In [88]:
train_preprocessd = []
for sample in tqdm(train_set[5900:6000]):
    sample = clean_data(sample)
    train_preprocessd.append(find_fake_answer(sample))


 43%|█████████████████▋                       | 43/100 [00:09<00:12,  4.61it/s]
Exception in thread Thread-15:
Traceback (most recent call last):
  File "D:\ProgramData\Anaconda3\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "D:\ProgramData\Anaconda3\lib\site-packages\tqdm\_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "D:\ProgramData\Anaconda3\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration
 55%|██████████████████████▌                  | 55/100 [00:09<00:08,  5.62it/s]


KeyboardInterrupt: 

In [93]:
sample = clean_data(train_set[5956])
# sample
sample_1 = find_fake_answer(sample)

In [92]:
len(sample['segmented_article_content'])

7

In [20]:
with open('./data/preprocessed.json', 'w', encoding='utf-8') as f:
        json.dump([sample_1,sample_1,sample_1], f)

In [14]:
with open('./data/preprocessed.json', 'r', encoding='utf-8') as f:
    d = json.load(f)

In [20]:
# from collections import defaultdict
# t = defaultdict(int)
t['b'] += 1

In [21]:
t

defaultdict(int, {'a': 3, 'b': 1})

In [12]:
def store_prerpocess_data():
    preprocess_data = []
    for i in range(1,201):
        with open('./data/preprocessed_%d.json' % i, 'r', encoding='utf-8') as f:
            d = json.load(f)
        preprocess_data.extend(d)
    with open('./data/preprocessed.json', 'w', encoding='utf-8') as f:
            json.dump(preprocess_data, f)

In [10]:
preprocess_data = []
for i in range(1,201):
    with open('./data/preprocessed_%d.json' % i, 'r', encoding='utf-8') as f:
        d = json.load(f)
    preprocess_data.extend(d)

In [13]:
with open('./data/preprocessed.json', 'w', encoding='utf-8') as f:
    json.dump(preprocess_data, f)

In [11]:
len(preprocess_data)

20000

In [5]:
preprocess_data[16599]

{'article_content': '最近十年来，美军不断深化“网络中心战”理论研究，并重视将理论研究成果应用于实践，指导部队建设和装备发展，有效推动了防务转型。美军在阿富汗和伊拉克的军事行动表明，这方面的努力已收到实际成效。目前，“网络中心战”理论与实践结合的问题也受到美国以外其他国家的重视，成为世界军事领域值得关注的一个新动向。     一、美军以“网络中心战”理论指导部队建设    “网络中心战”理论现已成为美军作战理论的主流思想和核心内容。2001年9月，美国国防部向国会递交了《网络中心战》报告，将“网络中心战”理论作为规范美军建设的指南。2003年、 2004年美国国防部又先后公布了两个版本的《网络中心战概念框架》，从“物理域”、“信息域”、“认知域”和“社会域”等多个视角对具备网络中心战能力的部队进行具体描述。  目前，“网络中心战”的基本原则涵盖了战略、战役、战术三个层面，总结并集成了作战、技术、管理、组织编成等诸多方面的理论研究成果和实践的初步经验。    1．“网络中心战”的宗旨是极大地提高一体化联合作战效能    “网络中心战”的实质是利用计算机信息网络对地理上分散的部队实施一体化的指挥控制，使联合作战效能实现数量级上的提高。换言之，即利用网络把各种探测器、武器系统、指挥控制系统有机联系在一起，对敌方实施快速、准确、连续有效的打击。具备网络中心战能力的军队，总是通过全面网络化，大幅提高信息共享水平，改进态势感知的共享质量，增强部队的协同和自我协调能力，提高指挥速度和持续作战能力，从而最终达到大幅度地提高作战行动的效率和效果。    2．“网络中心战”部队建设的九项原则    当前，美军“网络中心战”能力建设仍处于初级阶段。这个初级阶段的基本特征是在构建传感器网、交战网、信息网的同时，集中解决包括连通性、技术互操作性、一体化协同能力、一体化防护能力等在内的诸多关键能力的问题。    在此期间，美军通过理论探索和实践应用，提出构建“网络中心战”部队的9项原则。它们是：    ①信息优势第一原则  即通过适时、精确和相互关联的信息获取、处理和交流，形成信息优势；    ②共享感知原则  即将信息和知识按照需求等级转化为联合作战和协同作战各方所需的共同认识和态势感知；    ③快速指挥决策原则  即通过适当的方法与程序，迅速将信息优势转化为有利于快速

In [9]:
preprocess_data[16599]['segmented_article_content'][1]

['最近',
 '十年',
 '来',
 '，',
 '美军',
 '不断',
 '深化',
 '“',
 '网络',
 '中心',
 '战',
 '”',
 '理论',
 '研究',
 '，',
 '并',
 '重视',
 '将',
 '理论',
 '研究成果',
 '应用',
 '于',
 '实践',
 '，',
 '指导',
 '部队',
 '建设',
 '和',
 '装备',
 '发展',
 '，',
 '有效',
 '推动',
 '了',
 '防务',
 '转型',
 '。',
 '美军',
 '在',
 '阿富汗',
 '和',
 '伊拉克',
 '的',
 '军事行动',
 '表明',
 '，',
 '这方面',
 '的',
 '努力',
 '已',
 '收到',
 '实际',
 '成效',
 '。',
 '目前',
 '，',
 '“',
 '网络',
 '中心',
 '战',
 '”',
 '理论',
 '与',
 '实践',
 '结合',
 '的',
 '问题',
 '也',
 '受到',
 '美国',
 '以外',
 '其他',
 '国家',
 '的',
 '重视',
 '，',
 '成为',
 '世界',
 '军事',
 '领域',
 '值得',
 '关注',
 '的',
 '一个',
 '新动向',
 '。']

In [67]:
title = Counter([train_set[i]['article_title'] for i in range(20000)])

In [70]:
len(set(title))

18476

In [69]:
{x : title[x] for x in title if title[x] >= 2 }

{'沙特成功拦截一枚导弹 疑来自也门胡塞武装': 2,
 '美国驻黑山大使馆遭自杀式爆炸袭击 未造成严重损坏': 2,
 '王毅与坦桑尼亚外长马希加举行会谈': 3,
 '特雷莎梅：突访伊拉克劳军 造访中东搞军售': 2,
 '国家和军队多部门联合部署全面规范现役士兵职业技能鉴定': 2,
 '“J警报”称朝鲜疑将发弹道导弹!? NHK为误报致歉': 2,
 '伊拉克西部一咖啡馆遭爆炸袭击至少10人死亡': 2,
 '阿富汗首都发生巨大爆炸造成近400人伤亡': 2,
 '俄战略轰炸机向叙境内极端组织目标发射导弹': 3,
 '美国称“确信”叙利亚发动化武袭击 正研究化学制剂': 2,
 '习近平就阿富汗发生汽车炸弹袭击事件向加尼总统致慰问电': 2,
 '雪中送炭之举': 2,
 '叙政府谴责美空袭 以战机对叙目标发动导弹攻击': 2,
 '美三艘航母将齐聚西太搞军演 外媒：展示战力震慑朝鲜': 2,
 '朝鲜驻新大使馆：朝美会谈将顺利举行': 2,
 '王毅同朝鲜外相李勇浩举行会谈': 7,
 '美军核航母编队已到关岛 酝酿加大南海巡航力度': 2,
 '为防武力夺权 传中国武警部队将由中央军委直辖': 2,
 '外交部副部长孔铉佑同韩方谈朝鲜半岛局势': 3,
 '韩日首脑将在平昌举行会谈 或磋商慰安妇等议题': 2,
 '中国海军第二十七批护航编队结束对突尼斯访问': 2,
 '阿富汗西部遭塔利班袭击 造成至少20名警察身亡': 2,
 '沙特拦截两枚胡塞武装发射的导弹 击落两架无人机': 2,
 '一艘载有中国船员船只在马来西亚附近海域倾覆': 2,
 '中国将派雪豹特种部队打击叙境内“东突”分子？外交部回应': 2,
 '美军泄密变性士兵进哈佛 气走中情局新老领导': 2,
 '国防部回应美军机在吉布提受到中国驻吉基地激光照射': 2,
 '性能突出！美国海军接收第15艘弗吉尼亚级核潜艇': 2,
 '愤怒的抗议一边儿去 驻日美军新基地强行施工': 2,
 '简氏称东风26亮相阅兵式表明该导弹或已部署': 2,
 '俄罗斯检查站遭自杀式爆炸袭击 致警员1死2伤': 2,
 '习近平同冈比亚总统巴罗举行会谈': 7,
 '俄强化对招募恐怖分子罪行惩罚力度 最高可判无期徒刑': 2,
 '俄国防部网站遭到密集黑客攻击': 2,
 '八艘无人驾驶潜艇或将联合搜

In [66]:
for i, j in enumerate([preprocess_data[i]['article_title'] for i in range(4900)]):
    if j == '3名韩国人在加纳海域遭绑架 至今下落不明':
        print(i)

1958
2077


In [75]:
preprocess_data[2077]

{'article_content': '韩国外交部3月31日说，3名韩国人数天前在非洲加纳海域遭绑架，至今下落不明。 排水量500吨的渔船“海洋711”号搭载40多名船员，26日遭不明身份劫匪劫持。次日，在挟船驶往尼日利亚海域途中，劫匪把包括3名韩国人在内的少数船员转移至一艘快艇带走，至今下落不明。“海洋711”号28日返回加纳东部的特马港，大约40名加纳船员安全靠岸。 韩国外交部说，劫匪先前劫持两艘希腊船只未遂，从船上带走两名外国人。这两人据信与3名韩国公民同在一艘快艇上。韩方正与加纳、尼日利亚等国密切合作，以求确定遭绑架船员的下落。 韩国联合参谋本部说，3名韩国公民系遭尼日利亚海盗劫持。韩国军方已派遣在索马里亚丁湾执行任务的“文武大王”号军舰赶往搜寻，预计4月16日抵达事发海域。（闫洁）【新华社微特稿】',
 'article_id': '923',
 'article_title': '3名韩国人在加纳海域遭绑架 至今下落不明',
 'article_type': '新闻',
 'questions': [{'answer': '500吨',
   'answer_spans': [1, 2],
   'fake_answers': '500吨',
   'match_scores': 1.0,
   'most_related_para': 2,
   'question': '“海洋711”号 是排水量多少吨的渔船',
   'question_type': '数值型问题',
   'questions_id': '12789428-c83b-45e1-88eb-4a75fc3e54f5',
   'segmented_answer': ['500', '吨'],
   'segmented_question': ['“',
    '海洋',
    '711',
    '”',
    '号',
    ' ',
    '是',
    '排水量',
    '多少',
    '吨',
    '的',
    '渔船']},
  {'answer': '非洲加纳海域',
   'answer_spans': [14, 16],
   'fake_answers': '非洲加纳海域',
   'match_scores': 1.0,
   'm

In [74]:
preprocess_data[1958]

{'article_content': '\u3000\u3000韩国外交部3月31日说，3名韩国人数天前在非洲加纳海域遭绑架，至今下落不明。\u3000\u3000排水量500吨的渔船“海洋711”号搭载40多名船员，26日遭不明身份劫匪劫持。次日，在挟船驶往尼日利亚海域途中，劫匪把包括3名韩国人在内的少数船员转移至一艘快艇带走，至今下落不明。“海洋711”号28日返回加纳东部的特马港，大约40名加纳船员安全靠岸。\u3000\u3000韩国外交部说，劫匪先前劫持两艘希腊船只未遂，从船上带走两名外国人。这两人据信与3名韩国公民同在一艘快艇上。韩方正与加纳、尼日利亚等国密切合作，以求确定遭绑架船员的下落。\u3000\u3000韩国联合参谋本部说，3名韩国公民系遭尼日利亚海盗劫持。韩国军方已派遣在索马里亚丁湾执行任务的“文武大王”号军舰赶往搜寻，预计4月16日抵达事发海域。（闫洁）【新华社微特稿】',
 'article_id': '920',
 'article_title': '3名韩国人在加纳海域遭绑架 至今下落不明',
 'article_type': '新闻',
 'questions': [{'answer': '26',
   'answer_spans': [15, 15],
   'fake_answers': '26',
   'match_scores': 1.0,
   'most_related_para': 2,
   'question': '排水量500吨的渔船“海洋711”号搭载40多名船员，多少日遭不明身份劫匪劫持。',
   'question_type': '数值型问题',
   'questions_id': '8e3ac811-e065-4af9-be9d-1da15ffddf9a',
   'segmented_answer': ['26'],
   'segmented_question': ['排水量',
    '500',
    '吨',
    '的',
    '渔船',
    '“',
    '海洋',
    '711',
    '”',
    '号',
    '搭载',
    '40',
    '多名',
    '船员',
    '，',
    '多少',
    '日',
   

In [ ]:
class LESDataset(object):
    def __init__(self, train_file, test_file = None):
        self.train_set = self._load_dataset(train_file)
        
        if test_file:
            self.test_set = self._load_dataset(test_file)
    
    def _load_dataset(self, data_path):
        with open(data_path, 'r', encoding='utf8'):
            dataset = json.load(data_path)
        return dataset
    
    def word_iter(self, set_name=None):
        if set_name == 'train':
            dataset = self.train_set
        elif set_name == 'test':
            dataset = self.test

In [95]:
import numpy as np
np.arange(0, 10, 2)

array([0, 2, 4, 6, 8])